# Add Liquidity
Position $\alpha$ adds liquidity with $\Delta r_\alpha$ of asset $i$.

In [1]:
import os
import sys
sys.path.insert(0, os.path.abspath(os.path.join(os.path.abspath(''), '..')))

import inspect
from model.amm.omnipool_amm import add_risk_liquidity

### Constraints
Adding liquidity must leave prices $p_j^Q$ unchanged for all assets $j$. It must also leave $\frac{R_i}{S_i}$ unchanged.

### Requirements
$$
\begin{align}
s_\alpha &= 0\\
r_\alpha &\geq -\Delta r_\alpha\\
\Delta r_\alpha &< 0
\end{align}
$$

### Updating AMM state
$$
\Delta Q_i = Q_i \frac{\Delta R_i}{R_i}\\
$$
If $\frac{Q_i + \Delta Q_i}{Q} > \omega_i$, the transaction fails due to the weight cap on asset $i$.

$$
\begin{align}
\Delta S_i &= S_i \frac{\Delta R_i}{R_i}\\
\Delta R_i &= -\Delta r_\alpha\\
\end{align}
$$

Let $U$ be the asset index of the selected stablecoin in Omnipool.
$$
\begin{align}
\Delta T_i &= Q_i^+ \frac{R_U^+}{Q_U^+} - T_i\\
\Delta T &= \Delta T_i
\end{align}
$$

If $T + \Delta T > C$, the transaction fails due to violation of the overall TVL cap $C$.

### Updating agent state
$$
\begin{align}
\Delta s_\alpha &= \Delta S_i\\
p_\alpha &= p_i^Q
\end{align}
$$

In [2]:
print(inspect.getsource(add_risk_liquidity))

def add_risk_liquidity(
        old_state: dict,
        old_agents: dict,
        LP_id: string,
        delta_R: float,
        i: int
) -> tuple:
    """Compute new state after liquidity addition"""

    assert delta_R > 0, "delta_R must be positive: " + str(delta_R)
    assert i >= 0, "invalid value for i: " + str(i)

    new_state = copy.deepcopy(old_state)
    new_agents = copy.deepcopy(old_agents)

    # Token amounts update
    new_state['R'][i] += delta_R
    new_agents[LP_id]['r'][i] -= delta_R

    # Share update
    new_state['S'][i] *= new_state['R'][i] / old_state['R'][i]
    new_agents[LP_id]['s'][i] += new_state['S'][i] - old_state['S'][i]

    # HDX add
    delta_Q = old_state['P'][i] * delta_R
    new_state['Q'][i] += delta_Q

    # set price at which liquidity was added
    new_agents[LP_id]['p'][i] = price_i(new_state, i)

    return new_state, new_agents

